WGAN机器增强版本
===
WGAN=>Wasserstein Generative Adversarial Networks

# 1.原始GAN究竟出了什么问题
原始GAN中判别器要最小化如下损失函数，尽可能把真实样本分为正例，生成样本分为负例：
$$-\mathbb{E}_{x\sim P_r}[\log D(x)] - \mathbb{E}_{x\sim P_g}[\log(1-D(x))] \tag{1}$$
其中$P_r$是真实样本分布，$P_g$是由生成器产生的样本分布。对于生成器，Goodfellow一开始提出来一个损失函数，后来又提出了一个改进的损失函数，分别是
$$\begin{eqnarray}
\mathbb{E}_{x\sim P_g}[\log(1-D(x))]\\
\mathbb{E}_{x\sim P_g}[- \log D(x)]
\end{eqnarray}$$
后者在WGAN两篇论文中称为“the - log D alternative”或“the - log D trick”

## 1.1.问题一
**判别器越好，生成器梯度消失越严重**。

# 6.Wassertein生成对抗网络-WGAN(Wassertein Generative Adversarial Networks)
## 6.1.改进
1. 判别器最后一层去掉sigmoid
2. 生成器和判别器的loss不取log
3. 对更新后的权重强制截断到一定范围内，比如[-0.01，0.01]，以满足论文中提到的lipschitz连续性条件。
4. 论文中也推荐使用SGD， RMSprop等优化器，不要基于使用动量的优化算法，比如adam

在WGAN中，D的任务不再是尽力区分生成样本与真实样本，而是尽量拟合出样本间的Wasserstein距离，从分类任务转化成回归任务。而G的任务则变成了尽力缩短样本间的Wasserstein距离

## 6.2.WGAN的作用
1. WGAN理论上给出了GAN训练不稳定的原因，即交叉熵(JS散度)不适合衡量具有不相交部分的分布之间的距离，转而使用wassertein距离去衡量生成数据分布和真实数据分布之间的距离，理论上解决了训练不稳定的问题。
2. 解决了模式崩溃的(collapse mode)问题，生成结果多样性更丰富。
3. 对GAN的训练提供了一个指标，此指标数值越小，表示GAN训练的越差，反之越好。可以说之前训练GAN完全就和买彩票一样，训练好了算你中奖，没中奖也不要气馁，多买几注吧。

## 6.3.Wassertein距离
GAN中交叉熵(JS散度)不适合衡量生成数据分布和真实数据分布的距离，如果通过优化JS散度训练GAN会导致找不到正确的优化目标，所以，WGAN提出使用wassertein距离作为优化方式训练GAN，但是数学上和真正代码实现上还是有区别的，使用Wasserteion距离需要满足很强的连续性条件—lipschitz连续性，为了满足这个条件，作者使用了将权重限制到一个范围的方式强制满足lipschitz连续性，但是这也造成了隐患.Lipschitz限制是在样本空间中，要求判别器函数D(x)梯度值不大于一个有限的常数K，通过权重值限制的方式保证了权重参数的有界性，间接限制了其梯度信息

# 7.Wassertein增强型生成对抗网络(Wassertein Generative Adversarial Networks-Gradient Penalty)
WGAN-GP是WGAN之后的改进版，主要还是改进了连续性限制的条件，因为，作者也发现将权重剪切到一定范围之后，比如剪切到[-0.01,+0.01]后，发生了这样的情况，如下图左边表示<br/>
![images](../images/02_06_01_003.png)<br/>
发现大多数的权重都在-0.01 和0.01上，这就意味了网络的大部分权重只有两个可能数，对于深度神经网络来说不能充分发挥深度神经网络的拟合能力，简直是极大的浪费。并且，也发现强制剪切权重容易导致梯度消失或者梯度爆炸，梯度消失很好理解，就是权重得不到更新信息，梯度爆炸就是更新过猛了，权重每次更新都变化很大，很容易导致训练不稳定。梯度消失与梯度爆炸原因均在于剪切范围的选择，选择过小的话会导致梯度消失，如果设得稍微大了一点，每经过一层网络，梯度变大一点点，多层之后就会发生梯度爆炸 。为了解决这个问题，并且找一个合适的方式满足lipschitz连续性条件，作者提出了使用梯度惩罚(gradient penalty)的方式以满足此连续性条件，其结果如上图右边所示.由于是对每个batch中的每一个样本都做了梯度惩罚（随机数的维度是(batchsize，1)），因此判别器中不能使用batch norm,但是可以使用其他的normalization方法，比如Layer Normalization、Weight Normalization和Instance Normalization，但是训练结果中WGAN-GP生成的图片比较模糊